In [1]:
import os
import glob
import textwrap
import time

import langchain
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate, LLMChain
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

/home/ruthvik/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral")


In [6]:

loader = DirectoryLoader(
    "/home/ruthvik/Downloads",
    glob="./test.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
    use_multithreading=True
)

documents = loader.load()

100%|██████████| 1/1 [00:01<00:00,  1.91s/it]


In [7]:
len(documents)

52

In [8]:
documents[17].page_content

'Evacuate the building as soon as the alarm sounds and proceed to the \ndesignated evacuation zone.\nDo not re-enter the building or work area until you have been instructed to do so \nby the emergency responders.\nT o use a fire extinguisher, remember the acronym PASS:\nP - PULL safety pin from handle.\nA - AIM nozzle at base of fire.\nS - SQUEEZE the trigger handle.\nS - SWEEP from side to side\nMany people tend to shrug off fire safety and prevention tips. But you \nhave to remember that fires are unpredictable. They can strike anytime \nand anywhere, especially when you least expect it. Having basic fire \nsafety measures put in place both at home and work can one day be the \ndifference between life and death.\nRegardless of who you are and what you do, make sure you’re you follow \nthe fire safety and prevention precautions in your area. Review the fire \nprevention tips below together with your family and friends. By doing so, \nyou’ll help create a safer place for you and your 

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 0
)

texts = text_splitter.split_documents(documents)

In [15]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs = {"device": "cuda"}
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [21]:
vectordb = FAISS.load_local(
    folder_path = "/home/ruthvik/Downloads/stuff/vectordb", 
    embeddings = embeddings)

In [22]:
vectordb.similarity_search('fire diasaster')

[Document(page_content='If the fire is small (wastebasket sized or smaller) and you have been trained to \nuse a fire extinguisher, you may attempt to extinguish the fire. Make sure that \nyou have a safe exit from the fire area and use the buddy system. If you cannot \nput out the fire in 5 seconds, evacuate.\nOn your way out, warn others.\nMove away from fire and smoke. Close doors and windows behind you, if time \npermits.\nBefore opening a door, place the back of your hand on the door to check for \nheat.  If cool, slowly check the door handle for heat and proceed to exit. \nIf the door is hot, do not open it. Find or create an alternative exit (windows).  If \nno other exit is available and you only see smoke, open the door slowly, move to \na crawling position, staying low, and quickly crawl to an exit.', metadata={'source': '/home/ruthvik/Downloads/test.pdf', 'page': 16}),
 Document(page_content='● Keep flammable things like dishcloths and paper towels away from \nstovetops and 

In [23]:
prompt_template = """
This is a chat bot made to help individuals who are stuck in a disaster.
Ask for the users Name, location, phone number and their status
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template = prompt_template, 
    input_variables = ["context", "question"]
)

In [24]:
llm_chain = LLMChain(prompt=PROMPT, llm=llm)
llm_chain

LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nThis is a chat bot made to help individuals who are stuck in a disaster.\nAsk for the users Name, location, phone number and their status\nAnswer in the same language the question was asked.\nUse only the following pieces of context to answer the question at the end.\n\n{context}\n\nQuestion: {question}\nAnswer:'), llm=Ollama(model='mistral'))

In [25]:
retriever = vectordb.as_retriever(search_kwargs = {"k": 3, "search_type" : "similarity"})
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever, 
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

In [26]:
vectordb.max_marginal_relevance_search("i am stuck in a fire", k = 3)

[Document(page_content='If the fire is small (wastebasket sized or smaller) and you have been trained to \nuse a fire extinguisher, you may attempt to extinguish the fire. Make sure that \nyou have a safe exit from the fire area and use the buddy system. If you cannot \nput out the fire in 5 seconds, evacuate.\nOn your way out, warn others.\nMove away from fire and smoke. Close doors and windows behind you, if time \npermits.\nBefore opening a door, place the back of your hand on the door to check for \nheat.  If cool, slowly check the door handle for heat and proceed to exit. \nIf the door is hot, do not open it. Find or create an alternative exit (windows).  If \nno other exit is available and you only see smoke, open the door slowly, move to \na crawling position, staying low, and quickly crawl to an exit.', metadata={'source': '/home/ruthvik/Downloads/test.pdf', 'page': 16}),
 Document(page_content='them with all the necessary information. Depending on your insurance, \nhopefully, 

In [27]:
vectordb.similarity_search("i am stuck in a fire", k = 3)

[Document(page_content='If the fire is small (wastebasket sized or smaller) and you have been trained to \nuse a fire extinguisher, you may attempt to extinguish the fire. Make sure that \nyou have a safe exit from the fire area and use the buddy system. If you cannot \nput out the fire in 5 seconds, evacuate.\nOn your way out, warn others.\nMove away from fire and smoke. Close doors and windows behind you, if time \npermits.\nBefore opening a door, place the back of your hand on the door to check for \nheat.  If cool, slowly check the door handle for heat and proceed to exit. \nIf the door is hot, do not open it. Find or create an alternative exit (windows).  If \nno other exit is available and you only see smoke, open the door slowly, move to \na crawling position, staying low, and quickly crawl to an exit.', metadata={'source': '/home/ruthvik/Downloads/test.pdf', 'page': 16}),
 Document(page_content='map out the fastest and safest way to go outside in case a fire starts.\nConsider t

In [28]:
def wrap_text_preserve_newlines(text, width=700):
    lines = text.split('\n')

    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])
    
    sources_used = ' \n'.join(
        [
            source.metadata['source'].split('/')[-1][:-4] + ' - page: ' + str(source.metadata['page'])
            for source in llm_response['source_documents']
        ]
    )
    
    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [29]:
def llm_ans(query):
    start = time.time()
    
    llm_response = qa_chain.invoke(query)
    ans = process_llm_response(llm_response)
    
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

In [30]:
llm_ans("i am stuck in a big fire")     

" I'm sorry for the situation you're in. Stay calm and try to find the fastest and safest way out. If the fire is small, you may attempt to extinguish it using a fire extinguisher if you have been trained to do so and have a safe exit. However, if you cannot put out the fire within 5 seconds, evacuate the area and warn others as you leave. Move away from the fire and smoke, close doors behind you if time permits, and check doors for heat before opening them. If no other exit is available, open the door slowly and crawl to an exit while staying low.\n\nIf you are trapped, stay calm and try to get someone's attention by tapping on hard or metal parts of the structure. In a crowded place, drop, cover, and hold on to protect yourself from falling debris. Do not rush for doorways as others will have the same idea. Agree on an area outside the building where you and your family or colleagues will meet after a fire.\n\nRemember, your safety is the priority.\n\nSources: \ntest - page: 16 \ntes